In [6]:
! pip install yahoo-finance

    100% |████████████████████████████████| 81kB 4.3MB/s 
  Running setup.py bdist_wheel for yahoo-finance ... - \ done
  Stored in directory: /Users/joanna/Library/Caches/pip/wheels/1d/24/ea/f8d6afbf3e527b3e4c8fbc347695a0aa1be5e3a2dd74053e30
  Running setup.py bdist_wheel for simplejson ... - \ | / - done
  Stored in directory: /Users/joanna/Library/Caches/pip/wheels/e4/32/71/60b361b0d05433eb9d1dd3d47619931c08cc4e387dc494ad3c
Successfully built yahoo-finance simplejson
You are using pip version 8.1.1, however version 8.1.2 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [7]:
! pip install --upgrade pip

    100% |████████████████████████████████| 1.2MB 561kB/s 
  Found existing installation: pip 8.1.1
    Uninstalling pip-8.1.1:
      Successfully uninstalled pip-8.1.1


In [15]:
from yahoo_finance import Share
yahoo = Share('YHOO')
print ('price:', yahoo.get_price())
print (yahoo.get_change())
print (yahoo.get_volume())
print (yahoo.get_prev_close())
print (yahoo.get_open())
print (yahoo.get_avg_daily_volume())
print (yahoo.get_stock_exchange())
print (yahoo.get_market_cap())
print (yahoo.get_book_value())
print (yahoo.get_ebitda())
print (yahoo.get_dividend_share())
print (yahoo.get_dividend_yield())
print (yahoo.get_earnings_share())
print (yahoo.get_days_high())
print (yahoo.get_days_low())
print (yahoo.get_year_high())
print (yahoo.get_year_low())
print (yahoo.get_50day_moving_avg())
print (yahoo.get_200day_moving_avg())
print (yahoo.get_price_earnings_ratio())
print (yahoo.get_price_earnings_growth_ratio())
print (yahoo.get_price_sales())
print (yahoo.get_price_book())
print (yahoo.get_short_ratio())
print (yahoo.get_trade_datetime())
print (yahoo.get_historical('2014-04-25', '2014-04-29'))
print (yahoo.get_info())

price: 36.66
+0.16
7784765
36.50
36.46
15135200
NMS
34.79B
30.03
245.44M
None
None
-4.76
36.94
36.43
43.78
26.15
36.86
33.47
None
-317.39
7.17
1.22
2.46
2016-05-23 20:00:00 UTC+0000
[{'Open': '34.369999', 'Close': '35.830002', 'Symbol': 'YHOO', 'Date': '2014-04-29', 'High': '35.889999', 'Volume': '28736000', 'Low': '34.119999', 'Adj_Close': '35.830002'}, {'Open': '34.669998', 'Close': '33.990002', 'Symbol': 'YHOO', 'Date': '2014-04-28', 'High': '35.00', 'Volume': '31019200', 'Low': '33.650002', 'Adj_Close': '33.990002'}, {'Open': '35.029999', 'Close': '34.48', 'Symbol': 'YHOO', 'Date': '2014-04-25', 'High': '35.099998', 'Volume': '19401600', 'Low': '34.290001', 'Adj_Close': '34.48'}]
{'CompanyName': None, 'symbol': 'YHOO', 'end': '2016-05-24', 'start': '1996-04-12'}


In [20]:
import pandas as pd
import urllib
import re
from yahoo_finance import Share
#import pandas.io.data as iod
#from pandas_datareader import data
#Also import numpy and matplotlib. Because ....
#%matplotlib inline
#import numpy as np
#import matplotlib.pyplot as plt
#import matplotlib as mp
#import datetime as dt

#read in the csv for the list of ticker names
ticker = pd.read_csv('Ticker.csv')
variable = pd.read_csv('Variable.csv')
#print (ticker)
#for every name, extract all the available finance data from yahoo finance
i = 0
length = len(ticker)
while i < length:
    tcr = ticker.at[i,'Ticker']
    name = Share(tcr)
    ticker.at[i,'Price'] = name.get_price()
    ticker.at[i,'BookValue'] = name.get_book_value()
    ticker.at[i,'Ebitda'] = name.get_ebitda()
    ticker.at[i,'Volume'] = name.get_volume()
    ticker.at[i,'MVolume'] = name.get_avg_daily_volume()
    ticker.at[i,'DivShare'] = name.get_dividend_share()
    ticker.at[i,'DivYield'] = name.get_dividend_yield()
    ticker.at[i,'EarnShare'] = name.get_earnings_share()
    ticker.at[i,'EarnRatio'] = name.get_price_earnings_ratio()
    ticker.at[i,'EarnGRatio'] = name.get_price_earnings_growth_ratio()
    ticker.at[i,'PriceSales'] = name.get_price_sales()
    ticker.at[i,'PriceBook'] = name.get_price_book()
    ticker.at[i,'ShortRatio'] = name.get_short_ratio()
    
    
    url = "http://finance.yahoo.com/q/ks?s=" + tcr + "+Key+Statistics"
    htmlfile = urllib.request.urlopen(url)
    htmltext = htmlfile.read()
    htmltext = htmltext.decode('utf-8')
    
    regex = '<span id="yfs_j10_' + tcr + '">(.+?)</span>'
    pattern = re.compile(regex)
    MarketCap = re.findall(pattern,htmltext)
    ticker.at[i,'MarketCap'] = MarketCap[0]
    
    url = "http://finance.yahoo.com/q/is?s=" + tcr + "+Income+Statement&annual"
    htmlfile = urllib.request.urlopen(url)
    htmltext = htmlfile.read()
    htmltext = htmltext.decode('utf-8')
    
    end=htmltext.find('&nbsp;&nbsp;')
    TotalRevenue = htmltext[end-12:end]
    ticker.at[i,'TotalRevenue'] = TotalRevenue
      
    #http://finance.yahoo.com/d/quotes.csv?s=AAPL+GOOG+MSFT&f=nab
    j = 0
    lengthj = len(variable)
    while j < lengthj:
        sym = variable.at[j,'Sym']
        var = variable.at[j,'Var']
        url = "http://finance.yahoo.com/d/quotes.csv?s=" + tcr + "&f=" + sym
        htmlfile = urllib.request.urlopen(url)
        htmltext = htmlfile.read()
        htmltext = htmltext.decode('utf-8')
        ticker.at[i,var] = htmltext
        j+=1
    i+=1
    
#format the data to a new csv file
print(ticker)
#print(variable)
ticker.to_csv('out.csv')

  Ticker  Price BookValue Ebitda      Volume   MVolume DivShare DivYield  \
0    bac  14.68   23.12\n   0.00  96183749\n  99855104     0.20     1.38   
1    jpm  64.54   61.29\n   0.00  13839390\n  15852200     1.76     2.77   
2    wfc  49.20   34.62\n   0.00  16597709\n  17895400     1.52     3.12   
3      c  45.86   71.47\n   0.00  18976539\n  20546100     0.20     0.44   

  EarnShare EarnRatio      ...      TickerTrend 1yrTargetPrice    MoreInfo  \
0      1.27     11.54      ...            N/A\n        17.35\n  96183749\n   
1      5.90     10.94      ...            N/A\n        70.57\n  13839390\n   
2      4.07     12.09      ...            N/A\n        55.00\n  16597709\n   
3      5.00      9.17      ...            N/A\n        56.11\n  18976539\n   

  HoldingsValue HoldingsValue(Realtime)        52weekRange DaysValueChange  \
0         N/A\n                   N/A\n  "10.99 - 18.48"\n           N/A\n   
1         N/A\n                   N/A\n  "50.07 - 70.61"\n           N/A

In [2]:
ticker = pd.read_csv('Ticker.csv')
a = ticker.at[0,'Ticker']
#df = pd.DataFrame(index = [a,a], columns=['A','B','C'])
#df
#ticker


#c = ticker.T
#b = c.values.tolist()
print(a)
#bb = pd.DataFrame(index = b, columns=['A','B','C'])
#bb
#bb.at[b[0],'A'] = 'Hello'
cc = pd.DataFrame(index = list(range(len(ticker))), columns=['Ticker'])

ticker
ticker.at[0,'A'] = 'Hello'
ticker.at[0,'B'] = 13
ticker.at[1,'A'] = 14
ticker

bac


,Ticker,A,B
0,bac,Hello,13
1,jpm,14,NaN
2,wfc,NaN,NaN
3,c,NaN,NaN


In [36]:
import urllib.request
import time

stockstoPull = 'AMD', 'BAC', 'MSFT', 'TXN', 'GOOG'

def pullData(stock):
    fileLine = stock + '.txt'
    urltovisit = 'http://chartapi.finance.yahoo.com/instrument/1.0/'+stock+'/chartdata;type=quote;range=1y/csv'
    with urllib.request.urlopen(urltovisit) as f:
        sourceCode = f.read().decode('utf-8')
    splitSource = sourceCode.split('\n')

    for eachLine in splitSource:
        splitLine = eachLine.split(',') # <---(here ',' instead of '.')
        if len(splitLine) == 6: # <----( here, 6 instead of 5 )
            if 'values' not in eachLine:
                saveFile = open(fileLine,'a')
                linetoWrite = eachLine+'\n'
                saveFile.write(linetoWrite)

    print('Pulled', stock)
    print('...')
    time.sleep(.5)

if __name__=="__main__":
    for eachStock in stockstoPull:     
        pullData(eachStock)

Pulled AMD
...
Pulled BAC
...
Pulled MSFT
...
Pulled TXN
...
Pulled GOOG
...


In [10]:
variable.at[0,'Var']

'Ask'

In [ ]:
    j = 0
    lengthj = len(variable)
    while j < lengthj:
        sym = variable.at[j,'Sym']
        var = variable.at[j,'Var']
        url = "http://finance.yahoo.com/d/quotes.csv?s=" + tcr + "&f=" + sym
        htmlfile = urllib.request.urlopen(url)
        htmltext = htmlfile.read()
        htmltext = htmltext.decode('utf-8')
        ticker.at[i,var] = htmltext
        j+=1